# Visualización mediante dashboards

Una parte clave del análisis de datos es ser capaz de visualizar los resultados de forma clara y comprensible. En este capítulo del curso vamos a aprender a construir dashboards, páginas web interactivas, para visualizar información usando Python.

## ¿Qué es una página web?

Una página web es un documento que se muestra en un navegador, compuesto generalmente por HTML (estructura), CSS (estilo) y JavaScript (comportamiento). Nosotros no escribiremos directamente estos lenguajes, pero usaremos herramientas que los generan por nosotros.

## Componentes de una página web

En general, una página web tiene elementos visuales con los que las personas interactúan. Algunos componentes comunes que usaremos son:

- **Encabezados y texto**: Se usan para estructurar visualmente la información.
- **Dropdowns (menús desplegables)**: Permiten seleccionar una opción de una lista.
- **Botones**: Permiten ejecutar acciones al hacer clic.
- **Checkboxes**: Permiten activar o desactivar opciones.
- **Gráficos**: Representan visualmente los datos.

Estos componentes forman el layout de nuestra app.

## ¿Qué es Dash y por qué lo usamos?

Dash es un framework de Python para construir aplicaciones web interactivas sin necesidad de escribir código en HTML, CSS o JavaScript. Nos permite crear páginas web directamente desde Python, sin tener que escribir HTML ni JavaScript. Es útil para construir dashboards interactivos que respondan a acciones del usuario como seleccionar una ciudad, cambiar un rango de fechas, hacer clic en un botón, etc. Sus componentes principales son **Layouts** y **Callbacks**.

## Layout en Dash

En Dash, todos los elementos visibles de una app se organizan dentro del atributo `layout`. Cada componente se define como un elemento dentro de `html.Div` o algún otro contenedor.

```python
app.layout = html.Div([
    html.H1("Título"),
    dcc.Dropdown(...),
    html.Button(...),
    dcc.Graph(...)
])
```

Esto define qué ve el usuario y en qué orden aparecen los elementos.

## Ejemplo de múltiples componentes

Podemos tener un botón que modifica un texto, un checkbox que activa una opción y un dropdown que actualiza un gráfico. Todo eso se puede controlar desde Python.

```python
html.Div([
    dcc.Checklist(
        options=[{'label': 'Mostrar valores', 'value': 'show'}],
        value=[],
        id='checkbox'
    ),
    html.Button('Actualizar', id='boton', n_clicks=0),
    dcc.Dropdown(id='dropdown-ciudad', ...),
    dcc.Graph(id='grafico')
])
```


## ¿Qué es un Callback y cómo funciona?

Un callback es una función en Python que se ejecuta automáticamente cuando el usuario interactúa con la página. Por ejemplo, al seleccionar una ciudad en un dropdown, podemos actualizar un gráfico.

Los callbacks están decorados con `@app.callback(...)` y tienen:

- `Output(...)`: indica qué componente va a cambiar y qué propiedad.
- `Input(...)`: indica qué componente y propiedad activan el cambio.
- `State(...)` (opcional): permite usar el valor actual de un componente sin que active el callback.

```python
@app.callback(
    Output('salida', 'children'),
    Input('dropdown', 'value')
)
def actualizar(valor):
    return f'Seleccionaste: {valor}'
```


## Setear y guardar estado

Algunos componentes permiten guardar estado, como los botones (`n_clicks`) o las checkboxes (`value`). Esto permite construir lógicas más complejas como mostrar/ocultar elementos, cambiar entre vistas o controlar flujos de interacción.

## Interfaces interactivas para visualización

Cuando construimos una visualización interactiva, no solo nos interesa mostrar datos, sino también permitir que el usuario explore y filtre esa información de forma intuitiva. Para esto, utilizamos componentes visuales como menús, botones, controles de fecha, etc.

Existen distintos tipos de componentes que permiten construir una interfaz flexible y dinámica.

**Listas desplegables y selección de opciones**

Un menú desplegable permite seleccionarpor ejemplo una ciudad o país. Esto lo hacemos con el componente `dcc.Dropdown`, que se puede configurar para permitir una o varias selecciones. También podemos usar `dcc.RadioItems` si queremos limitar al usuario a una única opción, o `dcc.Checklist` si queremos que seleccione varias a la vez.

Todos estos componentes comparten una propiedad clave: el atributo `value`, que representa lo que el usuario eligió.

**Botones y acciones**

Los botones (`html.Button`) permiten que el usuario indique cuándo ejecutar una acción, como actualizar un gráfico. Se usan en conjunto con el atributo `n_clicks`, que cuenta cuántas veces ha sido presionado el botón. Esto nos permite ejecutar código solo cuando el usuario lo decide.

**Sliders y control numérico**

Para permitir que el usuario explore intervalos, por ejemplo un rango de temperaturas o una escala de tiempo, usamos `dcc.Slider` o `dcc.RangeSlider`. Estos componentes también tienen un atributo `value` que usamos para saber qué rango fue seleccionado.

**Fechas e intervalos temporales**

El componente `dcc.DatePickerRange` permite al usuario seleccionar un intervalo de fechas. Esto es útil cuando queremos que el gráfico se ajuste solo al periodo que el usuario quiere analizar.

**Mostrar resultados: gráficos y texto**

La forma más común de mostrar resultados es usando `dcc.Graph`, que permite renderizar visualizaciones complejas de Plotly. También podemos mostrar valores, mensajes o resultados usando `html.Div`, y si queremos mostrar una tabla de datos usamos `dash_table.DataTable`.

**Cómo se conectan: callbacks**

Todos estos elementos no funcionan por sí solos. Necesitamos una forma de decirle a la aplicación que "cuando el usuario cambie esto, actualiza aquello". Eso se logra con un **callback**, una función que conecta una entrada (como el valor de un dropdown o el clic en un botón) con una salida (como el contenido de un gráfico).

Por ejemplo, si el usuario selecciona una ciudad, podemos hacer que un gráfico cambie automáticamente para mostrar los datos de esa ciudad.

**Control de estado y lógica**

Además de entradas y salidas, podemos usar componentes como `dcc.Store` para guardar datos o configuraciones temporalmente en la sesión del usuario. Esto permite separar la lógica de carga de datos de la lógica visual, o evitar que se recalculen cosas innecesarias.

También podemos mostrar u ocultar componentes dinámicamente usando la propiedad `style={'display': 'none'}` o similar, controlada desde un callback.


# Dashboard interactivo con Dash

A modo de ejemplo, vamos a realizar dashboard completo para visualizar datos climáticos diarios por ciudad y país.

Incluye:
- Múltiples secciones en pestañas (tabs)
- Filtros por fecha, ciudad y variable
- Gráficos y métricas
- Uso de callbacks con Input, Output y State


## Cargando los datos

Vamos a trabajar con tres conjuntos de datos:
- `df_weather`: contiene registros climáticos como temperatura, precipitación y viento.
- `df_countries`: contiene datos como población, area y continente
- `df_cities`: contiene la información geográfica asociada a las estaciones y ciudades.

Los usaremos juntos para crear visualizaciones filtrables por país y ciudad.

Instalar los paquetes necesarios

In [ ]:
!pip install pyarrow fastparquet

In [ ]:
import pandas as pd

In [ ]:
# Cargar los datasets
df_weather = pd.read_parquet("daily_weather_clean.parquet", engine="pyarrow")
df_countries = pd.read_csv("countries.csv")
df_cities = pd.read_csv("cities.csv")

# Parsear fecha
df_weather["datetime"] = pd.to_datetime(df_weather["date"])
df_weather = df_weather.dropna()

### Explorando los datos a nivel superficial

In [ ]:
df_weather.columns

In [ ]:
df_countries.columns

In [ ]:
df_cities.columns

In [ ]:
df_weather

In [ ]:
df_countries

In [ ]:
df_cities

In [ ]:
# Merge con ciudades, sin duplicar 'city_name'
weather_cities = df_weather.merge(
    df_cities.drop(columns=["city_name"]),
    on='station_id'
)

# Renombrar columna 'country' en df_countries para evitar conflicto
df_countries_renamed = df_countries.rename(columns={"country": "country_name"})

# Merge con países
full_df = weather_cities.merge(
    df_countries_renamed.drop(columns=["iso3", "capital", "capital_lat", "capital_lng", "native_name"]),
    on='iso2'
)

# Filtrar y limpiar datos
df_weather_fullinfo = full_df.dropna(subset=["avg_temp_c", "city_name", "country_name", "date"])

In [ ]:
df_weather_fullinfo['year'] = pd.to_datetime(df_weather_fullinfo['date']).dt.year

In [ ]:
df_weather_fullinfo.columns

In [ ]:
df_weather_fullinfo[df_weather_fullinfo["country"]=="Germany"]["population"]

## Crear una app local con Dash

In [ ]:
!pip install dash dash-bootstrap-components

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64
import dash_bootstrap_components as dbc
from google.colab import output

Crear la app y nombrarla

In [ ]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Clima Global"

Preparar los datos a utilizar

In [ ]:
df = df_weather_fullinfo.copy()
variables = ["avg_temp_c", "precipitation_mm", "avg_wind_speed_kmh", "sunshine_total_min"]

paises = df['country'].unique()
ciudades_por_pais = {pais: df[df['country'] == pais]['city_name'].unique() for pais in paises}

image_filename = "icon.png" # Logo de la app
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

Una vez que tenemos los datos cargados, podemos construir interfaces que permitan al usuario explorar distintas dimensiones de la información. Por ejemplo, podemos permitir que seleccione un país y luego una ciudad para visualizar sus condiciones climáticas a lo largo del tiempo.

Este tipo de filtros se construyen usando componentes como Dropdown, Graph, y Callbacks que conectan la selección del usuario con los datos que se muestran.

## Crear el layout de la pagina

Lo primero es crear la estructura principal (layout), que contiene las distintas pestañas de la página junto con los elementos que la componen.

- **Home**: muestra una tarjeta con una introducción y estadísticas generales del conjunto de datos (número de registros, ciudades y países).
- **Explorador climático**: permite al usuario seleccionar un país, una ciudad, un rango de fechas y variables climáticas a visualizar. Al hacer clic en "Actualizar", se generan:
  - Un gráfico de líneas con las variables seleccionadas a lo largo del tiempo.
  - Tarjetas que muestran los valores promedio y máximo de cada variable.
  - Una tarjeta adicional con información geográfica de la ciudad seleccionada (región, continente, latitud, longitud).

El diseño se apoya en `dash-bootstrap-components` para lograr una interfaz ordenada y responsiva.
También se utiliza `base64` para incrustar el ícono dentro de la app sin depender de archivos externos.

In [ ]:
app.layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image), height="60px", style={"marginRight": "15px"}),
        html.H1("Dashboard Clima Global", style={"margin": 0})
    ], style={"display": "flex", "alignItems": "center", "justifyContent": "center", "marginBottom": "20px"}),

    dcc.Tabs([
        dcc.Tab(label='Home', children=[
            dbc.Container([
                dbc.Row([
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                                html.Div([
                                    html.Img(src='data:image/png;base64,{}'.format(encoded_image), height="80px", style={"marginRight": "15px"}),
                                    html.H3("Bienvenido al Dashboard de Clima Global", className="card-title", style={"margin": 0})
                                ], style={"display": "flex", "alignItems": "center", "marginBottom": "20px"}),

                                html.P("Este dashboard permite explorar datos climáticos diarios por ciudad, permitiendo comparar distintas variables ambientales históricas.",
                                      className="card-text"),
                                html.Hr(),
                                html.P(f"Número total de registros: {len(df):,}", className="card-text"),
                                html.P(f"Número de ciudades: {df['city_name'].nunique()}", className="card-text"),
                                html.P(f"Número de países: {df['country'].nunique()}", className="card-text"),
                                html.Hr(),
                                html.P("Fuente de datos: dataset procesado desde registros históricos de estaciones meteorológicas internacionales.")
                            ])
                        ], className="mt-4")
                    ], width=8)
                ], justify="center")
            ])
        ]),
        dcc.Tab(label='Explorador climático', children=[
            dbc.Row([
                dbc.Col([
                    html.Label("País:"),
                    dcc.Dropdown(
                        id='pais-dropdown',
                        options=[{'label': pais, 'value': pais} for pais in sorted(paises)],
                        value="United States of America" # Valor por defecto
                    ),
                    html.Label("Ciudad:"),
                    dcc.Dropdown(
                        id='ciudad-dropdown',
                        options=[],
                        value="Denver"
                    ),
                    html.Label("Rango de fechas:"),
                    dcc.DatePickerRange(
                        id='date-range',
                        start_date=df['date'].min(),
                        end_date=df['date'].max()
                    ),
                    html.Br(),
                    html.Label("Variables a visualizar:"),
                    dcc.Checklist(
                        id='variable-checklist',
                        options=[{'label': v, 'value': v} for v in variables],
                        value=["avg_temp_c"]
                    ),
                    html.Button("Actualizar", id="submit-button", n_clicks=0)
                ], width=8),

                dbc.Col(
                    html.Div(id="info-ubicacion"),
                    width=4
                )
            ], className="mb-4"),

            html.Div(id="metricas"),
            dcc.Graph(id="grafico")
        ])
    ])
])

## Callback para actualizar gráfico y métricas

En esta parte del proyecto agregamos interactividad al dashboard mediante `callbacks`, una de las piezas clave de Dash.

Un callback en Dash permite conectar los componentes visuales (como botones, dropdowns o gráficas) con funciones de Python que procesan los datos y actualizan la interfaz dinámicamente.

Vamos a definir dos callbacks:
1. Uno para actualizar dinámicamente el listado de ciudades en base al país seleccionado.
2. Otro para generar el gráfico, las métricas y la información geográfica según la selección del usuario.


---

### 1. `set_ciudades`
- **Input:** el valor seleccionado en el dropdown de país.
- **Output:** la lista de opciones del dropdown de ciudad.
- **Objetivo:** cada vez que el usuario selecciona un país, se filtran las ciudades disponibles en ese país y se actualiza el segundo dropdown.

---

### 2. `actualizar_vista`
- **Input:** valores seleccionados en los filtros de país, ciudad, rango de fechas y variables; más el clic en el botón "Actualizar".
- **Output:** tres componentes:
  - Un gráfico (`figure`) con las variables seleccionadas.
  - Un conjunto de tarjetas (`metricas`) con el promedio y el máximo de cada variable.
  - Una tarjeta (`info-ubicacion`) con la región, continente y coordenadas de la ciudad.

La lógica se basa en filtrar el `DataFrame` según los valores seleccionados y generar componentes visuales (`plotly.express`, `dbc.Card`) con los datos resultantes.

Este mecanismo permite que la interfaz se actualice de forma reactiva, sin recargar la página, ofreciendo una experiencia fluida y personalizada.


In [ ]:
@app.callback(
    Output('ciudad-dropdown', 'options'),
    Input('pais-dropdown', 'value')
)
def set_ciudades(pais):
    ciudades = df[df["country"] == pais]["city_name"].unique()
    return [{'label': ciudad, 'value': ciudad} for ciudad in sorted(ciudades)]

@app.callback(
    [Output("grafico", "figure"),
     Output("metricas", "children"),
     Output("info-ubicacion", "children")],
    Input("submit-button", "n_clicks"),
    State("ciudad-dropdown", "value"),
    State("pais-dropdown", "value"),
    State("date-range", "start_date"),
    State("date-range", "end_date"),
    State("variable-checklist", "value")
)
def actualizar_vista(n, ciudad, pais, start_date, end_date, vars_sel):
    if not vars_sel or not ciudad:
        return {}, "Seleccione una ciudad y al menos una variable.", ""

    dff = df[(df["city_name"] == ciudad) &
             (df["country"] == pais) &
             (df["date"] >= start_date) &
             (df["date"] <= end_date)]

    if dff.empty:
        return {}, "No hay datos disponibles para esa selección.", ""

    fig = px.line(dff, x="date", y=vars_sel, title=f"Variables en {ciudad}")

    metricas = [
        dbc.Card(
            dbc.CardBody([
                html.H5(var.replace("_", " "), className="card-title"),
                html.P(f"Promedio: {dff[var].mean():.2f}", className="card-text"),
                html.P(f"Máximo: {dff[var].max():.2f}", className="card-text")
            ]),
            className="m-2", style={"width": "18rem", "display": "inline-block"}
        )
        for var in vars_sel
    ]

    ubicacion = dff.iloc[0][["region", "continent", "latitude", "longitude"]]
    tarjeta_ubicacion = dbc.Card(
        dbc.CardBody([
            html.H5("Información geográfica", className="card-title"),
            html.P(f"Región: {ubicacion['region']}"),
            html.P(f"Continente: {ubicacion['continent']}"),
            html.P(f"Latitud: {ubicacion['latitude']:.2f}"),
            html.P(f"Longitud: {ubicacion['longitude']:.2f}")
        ]),
        className="m-2", style={"width": "18rem"}
    )

    return fig, html.Div(metricas), tarjeta_ubicacion

## Ejecutar aplicación

In [ ]:
if __name__ == "__main__":
  output.serve_kernel_port_as_iframe(8050, height='1200') #esto solo debe incluirse si se utiliza Google Colab
  app.run(port=8050, debug=False)